# 【問題1】クロスバリデーション

In [297]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [2]:
data = pd.read_csv('home-credit-default-risk/application_train.csv')
test = pd.read_csv('home-credit-default-risk/application_test.csv')

In [3]:
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

In [4]:
# trainデータとtestデータを合わせて前処理したいのでラベル付けしておく
data['label'] = 0
test['label'] = 1

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 123 entries, SK_ID_CURR to label
dtypes: float64(65), int64(42), object(16)
memory usage: 288.6+ MB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 122 entries, SK_ID_CURR to label
dtypes: float64(65), int64(41), object(16)
memory usage: 45.4+ MB


In [7]:
train_y = data['TARGET']

In [8]:
data.drop('TARGET', axis=1, inplace=True)

In [9]:
df = pd.concat([data, test])

In [10]:
df

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,label
0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0000,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.0000,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0000,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.0100,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [11]:
# 性別の欠損値を消す
df.drop(list(df[df['CODE_GENDER']=='XNA'].index), inplace=True)

In [12]:
# 'ORGANIZATION_TYPE'の列を落とす
df.drop('ORGANIZATION_TYPE', axis=1, inplace=True)

In [13]:
# カテゴリ数が2のものはlabel encodingする
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object':
        if len(list(df[col].unique())) <= 2:
            le.fit(df[col])
            df[col] = le.transform(df[col])

In [14]:
# カテゴリ数が３以上のものはone-hot encodingする
df = pd.get_dummies(df)

In [15]:
# 正の相関が最も強いDAYS_BIRTHを年齢に直す
df['DAYS_BIRTH'] = abs(df['DAYS_BIRTH'])/365
df.rename(columns={'DAYS_BIRTH': 'age'}, inplace=True)

In [16]:
# 前処理が終わったのでtrainデータとtestデータに分ける
train = df[df['label'] == 0]
test = df[df['label'] == 1]

In [17]:
train

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,age,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,...,AMT_REQ_CREDIT_BUREAU_YEAR,label,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,25.920548,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,1.0,2,2,10,0,0,0,0,0,0,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.0220,0.0198,0.0,0.0000,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.

In [18]:
train_y.drop(list(data[data['CODE_GENDER']=='XNA'].index), inplace=True)

In [19]:
train_y

0         1
1         0
2         0
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    1
307510    0
Name: TARGET, Length: 307507, dtype: int64

In [20]:
train = pd.concat([train, train_y], axis=1)

In [21]:
train.drop('label', axis=1, inplace=True)
test.drop('label', axis=1, inplace=True)

C:\Anaconda\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
corr_target = train.corr()['TARGET'].sort_values()

In [110]:
corr_target[:5]

EXT_SOURCE_3                           -0.178926
EXT_SOURCE_2                           -0.160471
EXT_SOURCE_1                           -0.155317
age                                    -0.078242
NAME_EDUCATION_TYPE_Higher education   -0.056593
Name: TARGET, dtype: float64

In [23]:
corr_target[-6:-1]

CODE_GENDER                    0.054710
DAYS_LAST_PHONE_CHANGE         0.055219
NAME_INCOME_TYPE_Working       0.057483
REGION_RATING_CLIENT           0.058901
REGION_RATING_CLIENT_W_CITY    0.060895
Name: TARGET, dtype: float64

In [77]:
#targetとの相関係数が上位５個づつを使う
negative_corr_list_5 = corr_target[:5]
positive_corr_list_5 = corr_target[-6:-1]

In [78]:
train_x = train[negative_corr_list_5.index]
test_x = test[negative_corr_list_5.index]

In [63]:
# 欠損値を平均値で埋める
train_x.fillna(train_x.mean(), inplace=True)
test_x.fillna(train_x.mean(), inplace=True)

C:\Anaconda\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [64]:
train_x.isnull().sum()

EXT_SOURCE_3    0
EXT_SOURCE_2    0
EXT_SOURCE_1    0
dtype: int64

In [66]:
preds_list = []
models_list = []
roc_auc_score_list = []
cv = KFold(n_splits=5, shuffle=True, random_state=0)
for tr_idx, va_idx in cv.split(train_x):
    tr_x = train_x.loc[tr_idx, :]
    va_x = train_x.loc[va_idx, :]
    tr_y = train_y[tr_idx]
    va_y = train_y[va_idx]
    lr = LogisticRegression(random_state=0)
    lr.fit(tr_x, tr_y)
    pred_kfold = lr.predict_proba(va_x)[:, 1]
    preds_list.append(pred_kfold)
    models_list.append(lr)
    roc_auc_score_list.append(roc_auc_score(va_y, pred_kfold))

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## 欠損値はないはずなのにエラーが出た。理由が分からない

In [309]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (tr_idx, va_idx) in enumerate(cv.split(train_x)):
    tr_x = train_x.loc[tr_idx, :]
    va_x = train_x.loc[va_idx, :]
    tr_y = train_y[tr_idx]
    va_y = train_y[va_idx]
print(tr_x)
print(va_x)
print(tr_y)
print(va_y)

        EXT_SOURCE_3  EXT_SOURCE_2  EXT_SOURCE_1        age  \
0           0.139376      0.262949      0.083037  25.920548   
2           0.729567      0.555912           NaN  52.180822   
3                NaN      0.650442           NaN  52.068493   
5           0.621226      0.354225           NaN  46.413699   
6           0.492060      0.724000      0.774761  37.747945   
...              ...           ...           ...        ...   
307506           NaN      0.681632      0.145570  25.553425   
307507           NaN      0.115992           NaN  56.917808   
307508      0.218859      0.535722      0.744026  41.002740   
307509      0.661024      0.514163           NaN  32.769863   
307510      0.113922      0.708569      0.734460  46.180822   

        DAYS_LAST_PHONE_CHANGE  NEW_TAR_BY_EDU_TYPE  NEW_TAR_BY_CODE_GENDER  \
0                      -1134.0             0.089399                0.101419   
2                       -815.0             0.089399                0.101419   
3     

## 分けることはできていそう。

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
tr_x, va_x, tr_y, va_y = train_test_split(train_x, train_y, test_size=0.25, random_state=0)

In [44]:
lr = LogisticRegression(random_state=0)

In [45]:
preds_list = []
models_list = []
lr.fit(tr_x, tr_y)
pred_kfold = lr.predict_proba(va_x)[:, 1]
preds_list.append(pred_kfold)
models_list.append(lr)

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [46]:
preds_list

[array([0.02666368, 0.04307052, 0.04827272, ..., 0.02535092, 0.13027346,
        0.02760329])]

## 同じデータを使ってもtrain_test_splitだとうまくいく。なぜ…

## 欠損値があっても動くlightgbmで試してみる

In [98]:
folds = KFold(n_splits=5, shuffle=True, random_state=0)
score_list = []
validation_preds = np.zeros(train_x.shape[0])
submission_preds = np.zeros(test_x.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x)):
    trn_x, trn_y = train_x.iloc[trn_idx], train_y.iloc[trn_idx]
    val_x, val_y = train_x.iloc[val_idx], train_y.iloc[val_idx]
    
    clf = LGBMClassifier()
    
    clf.fit(trn_x, trn_y, eval_set= [(trn_x, trn_y), (val_x, val_y)], eval_metric='auc', verbose=300, early_stopping_rounds=150)
    score_list.append(roc_auc_score(val_y, clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]))
    validation_preds[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
    submission_preds += clf.predict_proba(test_x, num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.760006	training's binary_logloss: 0.246276	valid_1's auc: 0.737424	valid_1's binary_logloss: 0.250948
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.760918	training's binary_logloss: 0.246182	valid_1's auc: 0.73377	valid_1's binary_logloss: 0.251326
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.762791	training's binary_logloss: 0.244784	valid_1's auc: 0.724253	valid_1's binary_logloss: 0.257001
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.761056	training's binary_logloss: 0.245228	valid_1's auc: 0.730966	valid_1's binary_logloss: 0.25491
Training until validation scores don't improve for 150 rou

## lightgbmだと動く。なぜ…

In [82]:
roc_auc_score(train_y, validation_preds)

0.7326854957484529

In [86]:
submission_preds

array([0.03249522, 0.09489264, 0.02063736, ..., 0.04685618, 0.04801226,
       0.14973965])

## なぜロジスティック回帰だとうまくいかなかったのかわからなかったです。
## 以下lightgbmで進めていきます。

# 【問題2】グリッドサーチ

In [87]:
from sklearn.model_selection import GridSearchCV

In [91]:
param_grid = {'num_leaves': [50, 100, 150], 'min_data_in_leaf ': [10, 20, 30], 'max_depth': [10, 15, 20]}

In [92]:
model = GridSearchCV(estimator=LGBMClassifier(), param_grid=param_grid, cv=5, scoring='roc_auc')
model.fit(train_x, train_y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [10, 15, 20],
                         'min_da

In [96]:
print('Best parameters: {}'.format(model.best_params_))
print('Best cross-validation: {}'.format(model.best_score_))

Best parameters: {'max_depth': 10, 'min_data_in_leaf ': 10, 'num_leaves': 50}
Best cross-validation: 0.7320078262902662


## ３つの中で最もよかったパラメータが'max_depth': 10, 'min_data_in_leaf ': 10, 'num_leaves': 50でした

# 【問題3】Kaggle Notebooksからの調査

## 有安さんが特徴量を３０００個作ったという話を聞いて特徴量を作っているものを調べたい。

- 下記のコードでorganization_type毎の収入の中央値を作成してそれをorganizationtypeの列に代入している(target encodingのような形)
inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(inc_by_org)

- 収入と月額の借入額の比率を新しい特徴量として使っている（Week4で調べたときは相関係数は低かった）
- 総借入額を月額の借入額で割って返済期間を特徴量とし使っている。
- 総借入額と収入の比率を新しい特徴量として使っている
- 雇用されてからの期間と年齢の比率を新しい特徴量として使っている。
- kaggleのnotebookではないが、カテゴリ変数target-encodingをするとうまくいったという記事があったのでやってみる。

### このコンペの一位の人は７００個特徴量を作ったらしく、このようなコンペでは特徴量の作成と選択がスコアを左右するらしく、あまりパラメータのチューニングはあまり行っていないらしい。

# 【問題4】高い汎化性能のモデル作成

In [306]:
data = pd.read_csv('home-credit-default-risk/application_train.csv')
test = pd.read_csv('home-credit-default-risk/application_test.csv')

In [307]:
## days_birthを絶対値をとって365割ること年齢にする
data['DAYS_BIRTH'] = abs(data['DAYS_BIRTH'])/365
data.rename(columns={'DAYS_BIRTH': 'age'}, inplace=True)
test['DAYS_BIRTH'] = abs(test['DAYS_BIRTH'])/365
test.rename(columns={'DAYS_BIRTH': 'age'}, inplace=True)

In [240]:
# 'ORGANIZATION_TYPE'毎の収入をつくる
inc_by_org = data[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

In [241]:
inc_by_org

ORGANIZATION_TYPE
Advertising               165600.0
Agriculture               126000.0
Bank                      157500.0
Business Entity Type 1    157500.0
Business Entity Type 2    157500.0
Business Entity Type 3    157500.0
Cleaning                  135000.0
Construction              180000.0
Culture                   157500.0
Electricity               157500.0
Emergency                 162000.0
Government                135000.0
Hotel                     135000.0
Housing                   135000.0
Industry: type 1          157500.0
Industry: type 10         180000.0
Industry: type 11         137227.5
Industry: type 12         157500.0
Industry: type 13         135000.0
Industry: type 2          157500.0
Industry: type 3          135000.0
Industry: type 4          157500.0
Industry: type 5          157500.0
Industry: type 6          139050.0
Industry: type 7          144000.0
Industry: type 8          162450.0
Industry: type 9          180000.0
Insurance                 157500.0
Ki

In [242]:
# 作ったカテゴリごとのものを代入して新しい列を作る
data['NEW_INC_BY_ORG'] = data['ORGANIZATION_TYPE'].map(inc_by_org)
test['NEW_INC_BY_ORG'] = test['ORGANIZATION_TYPE'].map(inc_by_org)

In [243]:
data['NEW_INC_BY_ORG']

0         157500.0
1         135000.0
2         135000.0
3         157500.0
4         162000.0
            ...   
307506    157500.0
307507    117000.0
307508    135000.0
307509    157500.0
307510    157500.0
Name: NEW_INC_BY_ORG, Length: 307511, dtype: float64

In [244]:
# 'NAME_EDUCATION_TYPE'についてはtarget-encodingしていく
tar_by_edu_type = data[['NAME_EDUCATION_TYPE', 'TARGET']].groupby('NAME_EDUCATION_TYPE').mean()['TARGET']

In [245]:
# これも代入して新しい列を作る
data['NEW_TAR_BY_EDU_TYPE'] = data['NAME_EDUCATION_TYPE'].map(tar_by_edu_type)
test['NEW_TAR_BY_EDU_TYPE'] = test['NAME_EDUCATION_TYPE'].map(tar_by_edu_type)

In [246]:
# 'CODE_GENDER'もtarget-encodingする
tar_by_CODE_GENDER = data[['CODE_GENDER', 'TARGET']].groupby('CODE_GENDER').mean()['TARGET']

In [247]:
# これも代入して新しい列を作る
data['NEW_TAR_BY_CODE_GENDER'] = data['CODE_GENDER'].map(tar_by_CODE_GENDER)
test['NEW_TAR_BY_CODE_GENDER'] = test['CODE_GENDER'].map(tar_by_CODE_GENDER)

In [248]:
# 'CODE_GENDER'もtarget-encodingする
tar_by_name_income_type = data[['NAME_INCOME_TYPE', 'TARGET']].groupby('NAME_INCOME_TYPE').mean()['TARGET']

In [249]:
# これも代入して新しい列を作る
data['NEW_TAR_BY_NAME_INCOME_TYPE'] = data['NAME_INCOME_TYPE'].map(tar_by_name_income_type)
test['NEW_TAR_BY_NAME_INCOME_TYPE'] = test['NAME_INCOME_TYPE'].map(tar_by_name_income_type)

In [250]:
# 'REGION_RATING_CLIENT_W_CITY'もtarget-encodingする
tar_by_region_rating = data[['REGION_RATING_CLIENT_W_CITY', 'TARGET']].groupby('REGION_RATING_CLIENT_W_CITY').mean()['TARGET']

In [251]:
# これも代入して新しい列を作る
data['REGION_RATING_CLIENT_W_CITY'] = data['REGION_RATING_CLIENT_W_CITY'].map(tar_by_region_rating)
test['REGION_RATING_CLIENT_W_CITY'] = test['REGION_RATING_CLIENT_W_CITY'].map(tar_by_region_rating)

In [252]:
# 使う特徴量をリスト化する
features = ['EXT_SOURCE_3', 'EXT_SOURCE_2', 'EXT_SOURCE_1', 'age', 'DAYS_LAST_PHONE_CHANGE', 'NEW_TAR_BY_EDU_TYPE',
            'NEW_TAR_BY_CODE_GENDER', 'NEW_TAR_BY_NAME_INCOME_TYPE', 'REGION_RATING_CLIENT_W_CITY', 'NEW_INC_BY_ORG']

In [253]:
train_x = data[features]
train_y = data['TARGET']
test_x = test[features]

In [254]:
train_x.isnull().sum()

EXT_SOURCE_3                    60965
EXT_SOURCE_2                      660
EXT_SOURCE_1                   173378
age                                 0
DAYS_LAST_PHONE_CHANGE              1
NEW_TAR_BY_EDU_TYPE                 0
NEW_TAR_BY_CODE_GENDER              0
NEW_TAR_BY_NAME_INCOME_TYPE         0
REGION_RATING_CLIENT_W_CITY         0
NEW_INC_BY_ORG                      0
dtype: int64

In [ ]:
# 欠損値を平均で埋める
train_x.fillna(train_x.mean(), inplace=True)

In [256]:
train_x.isnull().sum()

EXT_SOURCE_3                   0
EXT_SOURCE_2                   0
EXT_SOURCE_1                   0
age                            0
DAYS_LAST_PHONE_CHANGE         0
NEW_TAR_BY_EDU_TYPE            0
NEW_TAR_BY_CODE_GENDER         0
NEW_TAR_BY_NAME_INCOME_TYPE    0
REGION_RATING_CLIENT_W_CITY    0
NEW_INC_BY_ORG                 0
dtype: int64

In [ ]:
# パラメータは問題２で求めたものを使います
params = {'max_depth': 10, 'min_data_in_leaf ': 10, 'num_leaves': 50}

In [258]:
folds = KFold(n_splits=5, shuffle=True, random_state=0)
score_list = []
validation_preds = np.zeros(train_x.shape[0])
submission_preds = np.zeros(test_x.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x)):
    trn_x, trn_y = train_x.iloc[trn_idx], train_y.iloc[trn_idx]
    val_x, val_y = train_x.iloc[val_idx], train_y.iloc[val_idx]
    
    clf = LGBMClassifier(max_depth=10, min_data_in_leaf=10, num_leaves=50)
    
    clf.fit(trn_x, trn_y, eval_set= [(trn_x, trn_y), (val_x, val_y)], eval_metric='auc', verbose=300, early_stopping_rounds=150)
    score_list.append(roc_auc_score(val_y, clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]))
    validation_preds[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
    submission_preds += clf.predict_proba(test_x, num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.785537	training's binary_logloss: 0.239236	valid_1's auc: 0.735605	valid_1's binary_logloss: 0.24903
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.783684	training's binary_logloss: 0.238813	valid_1's auc: 0.73999	valid_1's binary_logloss: 0.251785
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.783598	training's binary_logloss: 0.238673	valid_1's auc: 0.736975	valid_1's binary_logloss: 0.252603
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.784713	training's binary_logloss: 0.238311	valid_1's auc: 0.735517	valid_1's binary_logloss: 0.253584
Training until validation scores don't improve for 150 rou

In [259]:
score_list

[0.7356049539890841,
 0.7399901610662829,
 0.7369745644318042,
 0.7355165791023295,
 0.739502282379626]

## 欠損値の処理をせずにlightgbmを使ってみる。他の前処理は同じ

In [260]:
data = pd.read_csv('home-credit-default-risk/application_train.csv')
test = pd.read_csv('home-credit-default-risk/application_test.csv')

In [261]:
data['DAYS_BIRTH'] = abs(data['DAYS_BIRTH'])/365
data.rename(columns={'DAYS_BIRTH': 'age'}, inplace=True)
test['DAYS_BIRTH'] = abs(test['DAYS_BIRTH'])/365
test.rename(columns={'DAYS_BIRTH': 'age'}, inplace=True)

In [262]:
inc_by_org = data[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
data['NEW_INC_BY_ORG'] = data['ORGANIZATION_TYPE'].map(inc_by_org)
test['NEW_INC_BY_ORG'] = test['ORGANIZATION_TYPE'].map(inc_by_org)

In [263]:
tar_by_edu_type = data[['NAME_EDUCATION_TYPE', 'TARGET']].groupby('NAME_EDUCATION_TYPE').mean()['TARGET']
data['NEW_TAR_BY_EDU_TYPE'] = data['NAME_EDUCATION_TYPE'].map(tar_by_edu_type)
test['NEW_TAR_BY_EDU_TYPE'] = test['NAME_EDUCATION_TYPE'].map(tar_by_edu_type)

In [264]:
tar_by_CODE_GENDER = data[['CODE_GENDER', 'TARGET']].groupby('CODE_GENDER').mean()['TARGET']
data['NEW_TAR_BY_CODE_GENDER'] = data['CODE_GENDER'].map(tar_by_CODE_GENDER)
test['NEW_TAR_BY_CODE_GENDER'] = test['CODE_GENDER'].map(tar_by_CODE_GENDER)

In [265]:
tar_by_name_income_type = data[['NAME_INCOME_TYPE', 'TARGET']].groupby('NAME_INCOME_TYPE').mean()['TARGET']
data['NEW_TAR_BY_NAME_INCOME_TYPE'] = data['NAME_INCOME_TYPE'].map(tar_by_name_income_type)
test['NEW_TAR_BY_NAME_INCOME_TYPE'] = test['NAME_INCOME_TYPE'].map(tar_by_name_income_type)

In [266]:
tar_by_region_rating = data[['REGION_RATING_CLIENT_W_CITY', 'TARGET']].groupby('REGION_RATING_CLIENT_W_CITY').mean()['TARGET']
data['REGION_RATING_CLIENT_W_CITY'] = data['REGION_RATING_CLIENT_W_CITY'].map(tar_by_region_rating)
test['REGION_RATING_CLIENT_W_CITY'] = test['REGION_RATING_CLIENT_W_CITY'].map(tar_by_region_rating)

In [267]:
features = ['EXT_SOURCE_3', 'EXT_SOURCE_2', 'EXT_SOURCE_1', 'age', 'DAYS_LAST_PHONE_CHANGE', 'NEW_TAR_BY_EDU_TYPE',
            'NEW_TAR_BY_CODE_GENDER', 'NEW_TAR_BY_NAME_INCOME_TYPE', 'REGION_RATING_CLIENT_W_CITY', 'NEW_INC_BY_ORG']

In [268]:
train_x = data[features]
train_y = data['TARGET']
test_x = test[features]

In [269]:
folds = KFold(n_splits=5, shuffle=True, random_state=0)
score_list_nan = []
validation_preds_nan = np.zeros(train_x.shape[0])
submission_preds_nan = np.zeros(test_x.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x)):
    trn_x, trn_y = train_x.iloc[trn_idx], train_y.iloc[trn_idx]
    val_x, val_y = train_x.iloc[val_idx], train_y.iloc[val_idx]
    
    clf = LGBMClassifier(max_depth=10, min_data_in_leaf=10, num_leaves=50)
    
    clf.fit(trn_x, trn_y, eval_set= [(trn_x, trn_y), (val_x, val_y)], eval_metric='auc', verbose=300, early_stopping_rounds=150)
    score_list_nan.append(roc_auc_score(val_y, clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]))
    validation_preds_nan[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
    submission_preds_nan += clf.predict_proba(test_x, num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.787723	training's binary_logloss: 0.238641	valid_1's auc: 0.737556	valid_1's binary_logloss: 0.248511
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.78575	training's binary_logloss: 0.238237	valid_1's auc: 0.740184	valid_1's binary_logloss: 0.251754
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.784929	training's binary_logloss: 0.238033	valid_1's auc: 0.736201	valid_1's binary_logloss: 0.252694
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.786623	training's binary_logloss: 0.237767	valid_1's auc: 0.735734	valid_1's binary_logloss: 0.253437
Training until validation scores don't improve for 150 ro

In [270]:
# 欠損値を平均で埋めた結果
score_list

[0.7356049539890841,
 0.7399901610662829,
 0.7369745644318042,
 0.7355165791023295,
 0.739502282379626]

In [271]:
# 欠損値をそのままにした結果
score_list_nan

[0.7375563183035323,
 0.7401842532769025,
 0.7362013017751311,
 0.7357341485042477,
 0.7404997895943402]

## 欠損値の処理をしてもしなくても結果は変わらなかった

## notebookに載っていた特徴量を作成して使用する

In [277]:
data['CREDIT_TERM'] = data['AMT_ANNUITY'] / data['AMT_CREDIT']
test['CREDIT_TERM'] = test['AMT_ANNUITY'] / test['AMT_CREDIT']

In [278]:
features = ['EXT_SOURCE_3', 'EXT_SOURCE_2', 'EXT_SOURCE_1', 'age', 'DAYS_LAST_PHONE_CHANGE', 'NEW_TAR_BY_EDU_TYPE',
            'NEW_TAR_BY_CODE_GENDER', 'NEW_TAR_BY_NAME_INCOME_TYPE', 'REGION_RATING_CLIENT_W_CITY', 'NEW_INC_BY_ORG', 'CREDIT_TERM']

In [280]:
train_x = data[features]
train_y = data['TARGET']
test_x = test[features]

In [281]:
folds = KFold(n_splits=5, shuffle=True, random_state=0)
score_list_nan = []
validation_preds_nan = np.zeros(train_x.shape[0])
submission_preds_nan = np.zeros(test_x.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x)):
    trn_x, trn_y = train_x.iloc[trn_idx], train_y.iloc[trn_idx]
    val_x, val_y = train_x.iloc[val_idx], train_y.iloc[val_idx]
    
    clf = LGBMClassifier(max_depth=10, min_data_in_leaf=10, num_leaves=50)
    
    clf.fit(trn_x, trn_y, eval_set= [(trn_x, trn_y), (val_x, val_y)], eval_metric='auc', verbose=300, early_stopping_rounds=150)
    score_list_nan.append(roc_auc_score(val_y, clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]))
    validation_preds_nan[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
    submission_preds_nan += clf.predict_proba(test_x, num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.806333	training's binary_logloss: 0.232532	valid_1's auc: 0.757215	valid_1's binary_logloss: 0.242912
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.804393	training's binary_logloss: 0.232201	valid_1's auc: 0.757924	valid_1's binary_logloss: 0.246743
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.804373	training's binary_logloss: 0.23208	valid_1's auc: 0.753876	valid_1's binary_logloss: 0.247725
Training until validation scores don't improve for 150 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.804848	training's binary_logloss: 0.232063	valid_1's auc: 0.754172	valid_1's binary_logloss: 0.248293
Training until validation scores don't improve for 150 ro

## 特徴量一つで大きくスコアが上がった

In [282]:
submission = pd.read_csv('home-credit-default-risk/sample_submission.csv')

In [285]:
submission['TARGET'] = submission_preds_nan

In [286]:
submission.to_csv('submission_first.csv', index=False)

## kaggleに提出した結果
## Private Score:0.75214

## さらに特徴量を増やす

In [288]:
data['DAYS_EMPLOYED_PERC'] = data['DAYS_EMPLOYED'] / data['age']
test['DAYS_EMPLOYED_PERC'] = test['DAYS_EMPLOYED'] / test['age']
data['INCOME_CREDIT_PERC'] = data['AMT_INCOME_TOTAL'] / data['AMT_CREDIT']
test['INCOME_CREDIT_PERC'] = test['AMT_INCOME_TOTAL'] / test['AMT_CREDIT']
data['INCOME_PER_PERSON'] = data['AMT_INCOME_TOTAL'] / data['CNT_FAM_MEMBERS']
test['INCOME_PER_PERSON'] = test['AMT_INCOME_TOTAL'] / test['CNT_FAM_MEMBERS']
data['ANNUITY_INCOME_PERC'] = data['AMT_ANNUITY'] / data['AMT_INCOME_TOTAL']
test['ANNUITY_INCOME_PERC'] = test['AMT_ANNUITY'] / test['AMT_INCOME_TOTAL']

In [289]:
features = ['EXT_SOURCE_3', 'EXT_SOURCE_2', 'EXT_SOURCE_1', 'age', 'DAYS_LAST_PHONE_CHANGE', 'NEW_TAR_BY_EDU_TYPE',
            'NEW_TAR_BY_CODE_GENDER', 'NEW_TAR_BY_NAME_INCOME_TYPE', 'REGION_RATING_CLIENT_W_CITY', 'NEW_INC_BY_ORG', 
            'CREDIT_TERM', 'DAYS_EMPLOYED_PERC', 'INCOME_CREDIT_PERC', 'INCOME_PER_PERSON', 'ANNUITY_INCOME_PERC']

In [290]:
train_x = data[features]
train_y = data['TARGET']
test_x = test[features]

## ハイパーパラメータをkaggleのnotebookにある、Bayesian Optimizationを使用して調整したものを使用する

In [292]:
folds = KFold(n_splits=5, shuffle=True, random_state=0)
score_list_nan = []
validation_preds_nan = np.zeros(train_x.shape[0])
submission_preds_nan = np.zeros(test_x.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x)):
    trn_x, trn_y = train_x.iloc[trn_idx], train_y.iloc[trn_idx]
    val_x, val_y = train_x.iloc[val_idx], train_y.iloc[val_idx]
    
    clf = LGBMClassifier(nthread=4,
   n_estimators=10000,
   learning_rate=0.02,
   num_leaves=34,
   colsample_bytree=0.9497036,
   subsample=0.8715623,
   max_depth=8,
   reg_alpha=0.041545473,
   reg_lambda=0.0735294,
   min_split_gain=0.0222415,
   min_child_weight=39.3259775,
   silent=-1,
   verbose=-1, )
    
    clf.fit(trn_x, trn_y, eval_set= [(trn_x, trn_y), (val_x, val_y)], eval_metric='auc', verbose=300, early_stopping_rounds=150)
    score_list_nan.append(roc_auc_score(val_y, clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]))
    validation_preds_nan[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
    submission_preds_nan += clf.predict_proba(test_x, num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

Training until validation scores don't improve for 150 rounds
[300]	training's auc: 0.775598	training's binary_logloss: 0.241898	valid_1's auc: 0.758587	valid_1's binary_logloss: 0.242812
[600]	training's auc: 0.789828	training's binary_logloss: 0.237209	valid_1's auc: 0.762983	valid_1's binary_logloss: 0.24155
[900]	training's auc: 0.80092	training's binary_logloss: 0.233615	valid_1's auc: 0.763742	valid_1's binary_logloss: 0.24135
[1200]	training's auc: 0.810031	training's binary_logloss: 0.230532	valid_1's auc: 0.763845	valid_1's binary_logloss: 0.241322
Early stopping, best iteration is:
[1093]	training's auc: 0.806897	training's binary_logloss: 0.231597	valid_1's auc: 0.763978	valid_1's binary_logloss: 0.241306
Training until validation scores don't improve for 150 rounds
[300]	training's auc: 0.775974	training's binary_logloss: 0.240955	valid_1's auc: 0.760192	valid_1's binary_logloss: 0.246083
[600]	training's auc: 0.790835	training's binary_logloss: 0.236056	valid_1's auc: 0.76

In [294]:
score_list_nan

[0.7639784618212879,
 0.7644119768887283,
 0.7591779431987098,
 0.7612267868059186,
 0.7625783803389731]

## 最もスコアがよかったのでこちらをkaggleに提出する

# 【問題5】最終的なモデルの選定

In [295]:
submission['TARGET'] = submission_preds_nan

In [296]:
submission.to_csv('submission_second.csv', index=False)

## kaggleに提出した結果
## Private Score:0.75727

## xgboostを試してみる

In [300]:
folds = KFold(n_splits=5, shuffle=True, random_state=0)
score_list_nan = []
validation_preds_nan = np.zeros(train_x.shape[0])
submission_preds_nan = np.zeros(test_x.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_x)):
    trn_x, trn_y = train_x.iloc[trn_idx], train_y.iloc[trn_idx]
    val_x, val_y = train_x.iloc[val_idx], train_y.iloc[val_idx]
    
    clf = XGBClassifier(learning_rate =0.01, n_estimators=10000, max_depth=4, min_child_weight=5, subsample=0.8, colsample_bytree=0.8, 
                        objective= 'binary:logistic', nthread=4, scale_pos_weight=2.5, seed=27, reg_lambda = 1.2)
    
    clf.fit(trn_x, trn_y, eval_set= [(trn_x, trn_y), (val_x, val_y)], eval_metric='auc', verbose=100, early_stopping_rounds=200)
    score_list_nan.append(roc_auc_score(val_y, clf.predict_proba(val_x)[:, 1]))
    validation_preds_nan[val_idx] = clf.predict_proba(val_x)[:, 1]
    submission_preds_nan += clf.predict_proba(test_x)[:, 1] / folds.n_splits

[0]	validation_0-auc:0.70427	validation_1-auc:0.70161
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[100]	validation_0-auc:0.73046	validation_1-auc:0.72569
[200]	validation_0-auc:0.73788	validation_1-auc:0.73178
[300]	validation_0-auc:0.74556	validation_1-auc:0.73867
[400]	validation_0-auc:0.75184	validation_1-auc:0.74451
[500]	validation_0-auc:0.75663	validation_1-auc:0.74878
[600]	validation_0-auc:0.76009	validation_1-auc:0.75162
[700]	validation_0-auc:0.76321	validation_1-auc:0.75410
[800]	validation_0-auc:0.76571	validation_1-auc:0.75598
[900]	validation_0-auc:0.76817	validation_1-auc:0.75765
[1000]	validation_0-auc:0.77017	validation_1-auc:0.75889
[1100]	validation_0-auc:0.77189	validation_1-auc:0.75982
[1200]	validation_0-auc:0.77354	validation_1-auc:0.76073
[1300]	validation_0-auc:0.77517	validation_1-auc:0.76156
[1400]	validation_0-auc:0.77653	validation_1-auc:0.76213

[4400]	validation_0-auc:0.80678	validation_1-auc:0.76289
Stopping. Best iteration:
[4210]	validation_0-auc:0.80531	validation_1-auc:0.76296

[0]	validation_0-auc:0.70308	validation_1-auc:0.70014
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[100]	validation_0-auc:0.73018	validation_1-auc:0.72387
[200]	validation_0-auc:0.73797	validation_1-auc:0.73080
[300]	validation_0-auc:0.74575	validation_1-auc:0.73773
[400]	validation_0-auc:0.75218	validation_1-auc:0.74303
[500]	validation_0-auc:0.75723	validation_1-auc:0.74710
[600]	validation_0-auc:0.76098	validation_1-auc:0.74990
[700]	validation_0-auc:0.76415	validation_1-auc:0.75219
[800]	validation_0-auc:0.76675	validation_1-auc:0.75378
[900]	validation_0-auc:0.76903	validation_1-auc:0.75519
[1000]	validation_0-auc:0.77088	validation_1-auc:0.75614
[1100]	validation_0-auc:0.77261	validation_1-auc:0.75700
[1200]	validation_0-auc:0.774

In [303]:
score_list_nan

[0.767266817716606,
 0.7672073336468677,
 0.7629596762631973,
 0.7649711243442994,
 0.7654495221027479]

In [304]:
submission['TARGET'] = submission_preds_nan

In [305]:
submission.to_csv('submission_third.csv', index=False)

## kaggleに提出した結果
## Private Score:0.75750

# 考察

## training's aucとvalid_1's aucの間に約0.04の差があり、汎化性能がよいモデルとはいえないモデルができてしまった。target-encodingしたことで過学習をおこしてしまったのだろうか…